In [17]:
import pandas as pd
# pd.set_option('display.max_rows', None)

In [18]:
window_size = 5

### 준비

In [19]:
transaction_df = pd.read_csv('../데이터/Transaction/transaction_all.csv')
transaction_df['계약연도'] = transaction_df['계약년월'].apply(lambda x: str(x)[:4])
transaction_df['거래금액(만원)'] = transaction_df['거래금액(만원)'].apply(lambda x: int(x.replace(',','')))
transaction_df['제곱미터당 거래금액(만원)'] = transaction_df['거래금액(만원)'] / transaction_df['전용면적(㎡)']
transaction_df['단지'] = transaction_df['시군구'] + ' ' + transaction_df['단지명']
transaction_df = transaction_df[['단지', '계약년월', '건축년도', '제곱미터당 거래금액(만원)']]

In [20]:
economy_df = pd.read_excel('../데이터/Economy/economy_all.xlsx')
date_range = pd.date_range(start="2006-01", periods=len(economy_df), freq="M")
economy_df['계약년월'] = date_range.strftime('%Y%m').astype(int)

In [21]:
print(len(transaction_df))
print(len(transaction_df['단지'].unique()))

1237568
8977


### 동일 동,단지이지만 건축년도가 다른 단지 제거

In [22]:
transaction_df = transaction_df.groupby('단지').filter(lambda x: x['건축년도'].nunique() == 1)

### 동일 단지,계약년월일 경우, 제곱미터당 거래금액(만원) 평균

In [23]:
transaction_df = transaction_df.groupby(['단지','계약년월','건축년도'])['제곱미터당 거래금액(만원)'].mean().to_frame().reset_index()

In [24]:
print(len(transaction_df))
print(len(transaction_df['단지'].unique()))

429210
8957


### 2022년도 전까지 거래량 적은(window_size+1 미만) 단지 제거

In [25]:
transaction_else_df = transaction_df[transaction_df['계약년월']//100 != 2022]
apartment_complex_volume = transaction_else_df.groupby('단지').agg({'건축년도':'first','제곱미터당 거래금액(만원)':'count'}).reset_index().rename(columns={'제곱미터당 거래금액(만원)':'거래량'})
small_volume_apartment_complexes = apartment_complex_volume[apartment_complex_volume['거래량'] < window_size+1]['단지'].to_list()
transaction_df = transaction_df[~transaction_df['단지'].isin(small_volume_apartment_complexes)].reset_index(drop=True)

In [26]:
print(len(transaction_df))
print(len(transaction_df['단지'].unique()))

426378
8124


### 이상치 제거

In [11]:
transaction_df['Z 스코어'] = transaction_df.groupby('단지')['제곱미터당 거래금액(만원)'].transform(lambda x: (x-x.mean()) / x.std())
transaction_df = transaction_df[transaction_df['Z 스코어'].between(-3, 3)]
transaction_df = transaction_df.drop(columns=['Z 스코어']).reset_index(drop=True)

In [12]:
print(len(transaction_df))
print(len(transaction_df['단지'].unique()))

419695
8083


### 2022에만 있는 단지 제거

In [13]:
transaction_2022_df = transaction_df[transaction_df['계약년월']//100 == 2022]
apartment_complex_only_2022 = set(transaction_2022_df['단지'].unique()) - set(transaction_else_df['단지'].unique())
transaction_df = transaction_df[~transaction_df['단지'].isin(apartment_complex_only_2022)]

In [14]:
print(len(transaction_df))
print(len(transaction_df['단지'].unique()))

419619
8061


### 동과 단지 분리 후 저장

In [15]:
transaction_df['동'] = transaction_df['단지'].apply(lambda x: x.split(' ')[:3]).apply(lambda x: ' '.join(x))
transaction_df['단지'] = transaction_df['단지'].apply(lambda x: x.split(' ')[3:]).apply(lambda x: ' '.join(x))
transaction_df = transaction_df[['동','단지','건축년도','계약년월','제곱미터당 거래금액(만원)']]

In [16]:
# transaction_df.to_excel('transaction_all.xlsx')
# transaction_df.to_csv('transaction_all.csv')